### ----------------------------------------------------------------------------------------------------------
## 社会変革型 医療データサイエンティスト育成講座
# Chapter 4: 生存時間分析とCox比例ハザードモデル
### ----------------------------------------------------------------------------------------------------------

In [ ]:
# データとライブラリの準備

from lifelines import datasets, KaplanMeierFitter
from lifelines.plotting import plot_lifetimes

# lifelinesに入っているデータセットを使用します
gbsg2_data = datasets.load_gbsg2()
gbsg2_data.head()

## German Breast Cancer Study Group 2
contains the observations of 686 women  
http://ugrad.stat.ubc.ca/R/library/ipred/html/GBSG2.html

| column name | variables |
|:---------|:----------|
| horTh | hormonal therapy, a factor at two levels no and yes |
| age | of the patients in years |
| menostat | menopausal status, a factor at two levels pre (premenopausal) and post (postmenopausal) |
| tsize | tumor size (in mm) |
| tgrade | tumor grade, a ordered factor at levels I < II < III |
| pnodes | number of positive nodes |
| progrec | progesterone receptor (in fmol) |
| estrec | estrogen receptor (in fmol) |
| time | recurrence free survival time (in days) | 
| cens | censoring indicator (0- censored, 1- event) | 

In [ ]:
# timeデータの可視化
import matplotlib.pyplot as plt
%matplotlib inline

time = gbsg2_data['time']
event = gbsg2_data['cens']

time_sorted = time.sort_values().values
plot_lifetimes(time_sorted[:50], event_observed=event[:50])
plt.xlabel('time (days)')
plt.show()

In [ ]:
# Kaplan-meier曲線の描画

# sklearnと同じくまずインスタンスの作成
kmf = KaplanMeierFitter()

# 生存時間を表すtimeデータと、
kmf.fit(time, event_observed=event)

kmf.plot(ci_show=False)
plt.title('Survival function')
plt.xlabel('time (days)')
plt.show()

In [ ]:
# ホルモン療法の有無で比較

ax = plt.subplot(111)

# ホルモン治療ありの被験者のインデックスを抽出します
therapy = (gbsg2_data["horTh"] == "yes")

# timeデータの中でホルモン治療ありのインデックスを指定します
kmf.fit(time[therapy], event_observed=event[therapy], label="with hormonal therapy")
kmf.plot(ax=ax,ci_show=False)

# ~therapyとすることで、ホルモン治療なしのインデックスを指定します
kmf.fit(time[~therapy], event_observed=event[~therapy], label="No therapy")
kmf.plot(ax=ax,ci_show=False)

plt.ylim(0, 1)
plt.xlabel('time (days)')
plt.title("Survival time in GBSG2 data")
plt.show()

In [ ]:
# log-rank検定

from lifelines.statistics import logrank_test

results = logrank_test(time[therapy], time[~therapy], event[therapy], event[~therapy], alpha=.99)
results.print_summary()

In [ ]:
# cox比例ハザードモデル
import pandas as pd
from lifelines import CoxPHFitter

# インスタンスを作成します
cph = CoxPHFitter()

# カテゴリ変数をダミー化します
gbsg2_data_dm = pd.get_dummies(gbsg2_data,columns=['horTh','menostat','tgrade'],drop_first=True)

cph.fit(gbsg2_data_dm, duration_col='time', event_col='cens', show_progress=True)
cph.print_summary()
cph.plot()

In [ ]:
# （参考） Schoenfeld残差を用いた比例ハザード性の検証
cph.check_assumptions(gbsg2_data_dm, p_value_threshold=0.05, show_plots=False)